In [2]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelBinarizer
import pickle

In [3]:
def time_start(name_file):
    file_tsv = name_file[:-3]+'tsv'
    # Đọc tệp TSV bằng pandas
    # print(file_tsv)
    df = pd.read_csv(file_tsv, delimiter='\t')
    df=df.iloc
    df0 = df[:,0]
    df2 = df[:,2]
    for i in range(len(df2)):
        if(df2[i]==1):
            time = df0[i]
            break
    return df0[i]

In [4]:
### Chia file.wav ra moi folder

# data = 'training_data'

# out_file = 'training_data_all/'
# for file in os.listdir(data):
#     if(file.endswith(".wav") == True):

#         name_file = os.path.join(data,file)
#         time_begin = (time_start(name_file))
#         file_txt = (name_file.split('_')[0])+'_'+(name_file.split('_')[1])+'.txt'
#         # print(file_txt)
#         df = pd.read_csv(file_txt)
#         df=np.array(df)
#         # print(df)
#         if (df == '#Murmur: Absent').any().any():
#             # print('Absent')
#             shutil.copy(name_file, out_file +'Absent/'+str(time_begin)+'_'+file)
#         elif (df == '#Murmur: Present').any().any():
#             # print('Present')
#             shutil.copy(name_file, out_file +'Present/'+str(time_begin)+'_'+file)
#         elif (df == '#Murmur: Unknown').any().any():
#             # print('Unknown')
#             shutil.copy(name_file, out_file +'Unknown/'+str(time_begin)+'_'+file)

In [5]:
""" 
/training_data_all
        --->Unknown
        --->Absent
        --->Present
"""

' \n/training_data_all\n        --->Unknown\n        --->Absent\n        --->Present\n'

In [6]:
def show_spectrogram (file):
    audio, sr = librosa.load(file,sr=4000)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr,n_fft= 200 ,n_mels=64,hop_length=300)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram ,ref=np.max)

    fig, ax = plt.subplots(figsize=(10,4))
    a =librosa.display.specshow(mel_spectrogram_db,
                            x_axis='time',
                            y_axis='linear',
                            sr=sr,
                            hop_length=512,
                            )
    plt.title("Audio Spectrogram Representation")
    plt.colorbar(format='%+2.0f dB')
    plt.xlabel("Time(s)")
    plt.ylabel("Frequency (Hz)")
    plt.show()


In [7]:
# audio_file_absent = "training_data_all/Absent/0.02025_49745_TV.wav"
# audio_file_present = "training_data_all/Present/0.08025_85031_AV.wav"
# audio_file_unknown = "training_data_all/Unknown/0.720351_49761_MV.wav"
# show_spectrogram(audio_file_absent)
# show_spectrogram(audio_file_present)
# show_spectrogram(audio_file_unknown)

In [8]:
# data_train = 'training_data_all'
# data_time = []
# for folders in os.listdir(data_train):
#     print(folders)
#     name_folders = os.path.join(data_train,folders)
#     for file in os.listdir(name_folders):
#         name_files = os.path.join(name_folders,file)
#         y,sr = librosa.load(name_files,sr=4000)
#         audio_duration = (len(y)/sr)
#         data_time.append(audio_duration)

In [9]:
# np.min(data_time)


In [10]:
def create_data (audio,file,sr):
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr,n_fft= 256 ,n_mels=128,hop_length=157)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram ,ref=np.max)
    # mel_spectrogram_db = mel_spectrogram_db.reshape([128,128,1])
    x_data.append(mel_spectrogram_db)


In [11]:
def split_audio(name_files,file,time_limit,sr):
    audio, sr = librosa.load(name_files, sr=sr)
    file = file[:-4]
    segment_length = time_limit * sr
    num_segments = (len(audio) // segment_length)
    for i in range(num_segments):
        start_sample = i * segment_length
        end_sample = (i + 1) * segment_length

        segment = audio[start_sample:end_sample]
        create_data(segment,file,sr)
        label = name_files.split('/')[1]
        get_label(label)

In [12]:
def get_label(label):
    # print(folders)
    y_data.append(label)

In [13]:
def get_date(name_files,file,time_limit,sr = 4000):
    # print(name_files)
    split_audio(name_files,file,time_limit,sr)

In [74]:
data_train = 'training_data_all'
x_data = []
y_data = []
for folders in os.listdir(data_train):
    print(folders)
    name_folders = os.path.join(data_train,folders)
    for file in os.listdir(name_folders):
        name_files = os.path.join(name_folders,file)
        get_date(name_files,file,time_limit = 5,sr = 4000)


file = open('data_file.csv','wb')
pickle.dump((x_data,y_data),file)
file.close()

Unknown
Absent
Present


In [75]:
file = open('data_file.csv', 'rb')
data = pickle.load(file)
x_data,y_data =data
file.close()
x_data=np.array(x_data)
y_data = np.array(y_data)

In [76]:
from collections import Counter

counters = Counter(y_data)

for value, count in counters.items():
    print(f"{value} xuất hiện {count}")


Unknown xuất hiện 622
Absent xuất hiện 9865
Present xuất hiện 2453


In [1]:
x_data=np.array(x_data)
y_data=np.array(y_data)
print(x_data.shape)
print(y_data.shape)

NameError: name 'np' is not defined

In [57]:
x_test = []
y_test = []
x_train = []
y_train = []

In [58]:
encoder = LabelBinarizer()
y_data=encoder.fit_transform(y_data)
x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size=0.3,random_state=68)

In [59]:
# x_test = np.array(x_test)
# y_test = np.array(y_test)
# x_train = np.array(x_train)
# y_train = np.array(y_train)

In [60]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
input_shape = (128, 128)


model = Sequential()

model.add(Conv1D(64, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
model.add(BatchNormalization())

model.add(Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
model.add(BatchNormalization())

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
model.add(BatchNormalization())

model.add(LSTM(128,return_sequences=True))
model.add(LSTM(64, return_sequences=False))  


model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax')) 


model.compile(optimizer='adam', loss='MSE', metrics=['accuracy'])
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 128, 64)           24640     
                                                                 
 max_pooling1d_12 (MaxPooli  (None, 64, 64)            0         
 ng1D)                                                           
                                                                 
 batch_normalization_12 (Ba  (None, 64, 64)            256       
 tchNormalization)                                               
                                                                 
 conv1d_13 (Conv1D)          (None, 64, 128)           24704     
                                                                 
 max_pooling1d_13 (MaxPooli  (None, 32, 128)           0         
 ng1D)                                                           
                                                      

In [61]:

history = model.fit(x_train,y_train,batch_size=32, epochs=10,validation_data=(x_test, y_test), callbacks=[model_checkpoint])

2023-10-30 05:17:22.282021: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 593625088 exceeds 10% of free system memory.
2023-10-30 05:17:32.614867: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 566.12MiB (rounded to 593625088)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-10-30 05:17:32.614898: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-10-30 05:17:32.614907: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 80, Chunks in use: 80. 20.0KiB allocated for chunks. 20.0KiB in use in bin. 7.4KiB client-requested in use in bin.
2023-10-30 05:17:32.614912: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 26, Chunks in use: 26. 13

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.